# Gomoku

### Configure notebook

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np

tf.keras.backend.set_floatx("float64")
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Set fontsizes in figures
#params = {'legend.fontsize': 'large',
#          'axes.labelsize': 'large',
#          'axes.titlesize': 'large',
#          'xtick.labelsize': 'large',
#          'ytick.labelsize': 'large',
#          'legend.fontsize': 'large',
#          'legend.handlelength': 2}
#plt.rcParams.update(params)

In [16]:
class Game:
    
    def __init__(self, agents, N, visualize = False):
        
        self.agents = agents
        self.N = N
        self.visualize = visualize
        
        if self.visualize == True:
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(111)
            self.fig.show()
            self.fig.canvas.draw()
            
            self.state = np.zeros((2,self.N,self.N), dtype = int)
            self.turn = 0
            self.X = [[np.copy(self.state)],[]]
            self.y = [[],[]]

    def count_in_a_row(self, i, move, x, y):
        
        in_a_row = 1
        counting = True
        j, k = move
        
        while(counting and -1 < j+x < 15 and -1 < k+y < 15):
            if self.state[i, j+x, k+y] == 1:
                in_a_row += 1
                j += x
                k += y
            else:
                counting = False
        
        counting = True
        j, k = move
        
        while(counting and -1 < j-x < 15 and -1 < k-y < 15):
            if self.state[i, j-x, k-y] == 1:
                in_a_row += 1
                j -= x
                k -= y
            else:
                counting = False
            
        return in_a_row
    

    def check_win(self, i, move):
        j, k = move
        
        if self.count_in_a_row(i, move, 1, 0) >= 5:
            return True
            
        if self.count_in_a_row(i, move, 0, 1) >= 5:
            return True
        
        if self.count_in_a_row(i, move, 1, 1) >= 5:
            return True
            
        if self.count_in_a_row(i, move, 1, -1) >= 5:
            return True
        
        return False
            
        
    def next_move(self):
        i = self.turn % 2
        
        if i == 0:
            move = self.agents[i](self.state)
        else:
            move = self.agents[i](self.state[::-1])

        self.X[i].append(np.copy(self.state))
        self.y[i].append(self.N*move[0] + move[1])
            
        self.state[i][move] = 1
        self.turn += 1
        
        return self.check_win(i, move)
       

    def play(self):
        self.state = np.zeros((2, 15, 15), dtype = int)
        self.turn = 0
        self.X = [[],[]]
        self.y = [[],[]]
        
        while(not self.next_move()):
            pass
        
        self.X = np.array(self.X)
        self.y = np.array(self.y)
        
     
    def plot(self):
        player1 = np.where(self.state[0] == 1)
        player2 = np.where(self.state[1] == 1)

        self.ax.clear()   
        self.ax.plot(player1[0]+0.25, player1[1]+0.25, "ro")
        self.ax.plot(player2[0]+0.25, player2[1]+0.25, "bo")
        
        minorLocator = MultipleLocator(0.5)
        self.ax.yaxis.set_minor_locator(minorLocator)
        self.ax.xaxis.set_minor_locator(minorLocator)
        self.ax.grid(which = 'both')
        
        self.ax.axis([0, self.N, 0, self.N])
        self.fig.canvas.draw()
             

def user_input(states):
    x = int(input())
    y = int(input())
    return (x, y)


In [17]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2, 15, 15)),
    tf.keras.layers.Dense(units=450, activation='relu'),
    tf.keras.layers.Dense(units=225, activation='softmax')
])

In [25]:
%matplotlib notebook
#plt.ion()
def agent(state):
    moves = model.predict(state[np.newaxis, :])[0]
    
    while(True):
        move = np.argmax(moves)
        if state[0].ravel()[move] == 1 or state[1].ravel()[move] == 1:
            moves[move] = 0
        else:
            break
        
    move = np.unravel_index(move, (15, 15))
    return move


@tf.function
def loss(model, X_win, y_win):
    y_pred = model.predict(X_win)
    return tf.losses.MSE(y_pred, y_win)

@tf.function
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)


optimizer = tf.optimizers.ADAM(learning_rate=0.01)

optimizer.apply_gradients(zip(grads, model.trainable_variables))

loss_value, grads = grad(model, X_win, y_win)



game = Game([agent, agent], 15)

N = 100
for i in range(N):
    game.play()
    

#while not game.next_move():
#    game.plot()
    
#game.plot()    
#if game.turn % 2 == 1:
#    print("Red wins!")
#else:
#    print("Blue wins!")

#print(len(game.y[0]))

In [22]:
print(len(game.y[1]))

50
